In [ ]:
!pip install face_recognition
!pip install imutils==0.5.4
import face_recognition
import cv2
from imutils.video import VideoStream
import time

img = cv2.imread("my_face.jpg")
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encoding = face_recognition.face_encodings(rgb_img)[0]

# Use VideoStream to capture video from the webcam
video_capture = VideoStream(src=1).start()

time.sleep(2.0)  # Allow the camera sensor to warm up

frame_skip = 3  # Process every 3rd frame
frame_count = 0

# Variables for FPS calculation
start_time = time.time()
fps_counter = 0

while True:
    frame = video_capture.read()

    frame_count += 1

    # Skip frames if needed
    if frame_count % frame_skip != 2:
        continue

    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Check if the face matches your encoded face
        match = face_recognition.compare_faces([img_encoding], face_encoding)

        name = "Unknown"

        if match[0]:
            name = "Ya Boi Gabe"

        # Draw a rectangle around the face and display the label
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Update FPS counter
    fps_counter += 1

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Calculate and print FPS every 5 seconds
    if time.time() - start_time >= 5:
        fps = fps_counter / (time.time() - start_time)
        print(f"FPS: {fps}")
        start_time = time.time()
        fps_counter = 0

# Release the webcam and close all windows
video_capture.stop()
cv2.destroyAllWindows()
